In [ ]:
!pip install pytorch-lightning

In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor


In [ ]:

# Data Resizing with appropriate normalization for grayscale MNIST
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

full_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)



In [ ]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)




In [ ]:
# Define the LitVGG13 model tailored for MNIST
class LitVGG13(pl.LightningModule):
    def __init__(self, num_classes=10):
        super().__init__()
        # Initialize the VGG13 model
        self.model = models.vgg13(pretrained=False)
        # Adjust the first convolution layer for 1-channel grayscale input
        self.model.features[0] = nn.Conv2d(1, 64, kernel_size=3, padding=1)

        # Adjust the classifier for MNIST's 10 classes
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.forward(images)
        loss = self.loss_fn(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.forward(images)
        loss = self.loss_fn(outputs, labels)
        _, preds = torch.max(outputs, 1)
        acc = torch.sum(preds == labels).item() / len(preds)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self.forward(images)
        loss = self.loss_fn(outputs, labels)
        _, preds = torch.max(outputs, 1)
        acc = torch.sum(preds == labels).item() / len(preds)
        self.log('test_loss', loss)
        self.log('test_acc', acc)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        return [optimizer], [lr_scheduler]



In [7]:
trainer = pl.Trainer(
    max_epochs=1,
    enable_progress_bar=True,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=3),
        ModelCheckpoint(dirpath='checkpoints', monitor='val_acc', mode='max', save_top_k=1),
        LearningRateMonitor(logging_interval='epoch')
    ],
    logger=pl.loggers.CSVLogger(save_dir='logs'),
    accelerator='auto',
    devices=1
)


# Initialize the model
model = LitVGG13(num_classes=10)

# Train the model
trainer.fit(model, train_loader, val_loader)

# Test the model
test_result = trainer.test(model, dataloaders=test_loader)
print(test_result)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for perfor

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9672999978065491     │
│         test_loss         │    0.10416961461305618    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.10416961461305618, 'test_acc': 0.9672999978065491}]


In [10]:
import os
os.makedirs('models/MNIST', exist_ok=True)


In [11]:
torch.save(model.state_dict(), 'models/MNIST/vgg13_mnist.pth')


In [12]:
from google.colab import files
files.download('models/MNIST/vgg13_mnist.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>